# Transports méridiens de chaleur et de quantité de mouvement

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021**

Les fichiers de données au format netcdf (données quotidiennes ERA5 de la température, du vent zonal et du vent méridien pour l'année 2021) doivent être téléchargés et placés dans le répertoire data :

- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form

Il faut télécharger des données tri-horaires puis calculer les moyennes quotidiennes via la commande CDO daymean :

- https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-930002.2.6

--> fichiers era5_t_daily_2021.nc, era5_u_daily_2021.nc, era5_v_daily_2021.nc

Fichiers à retraiter avec CDO (https://code.mpimet.mpg.de/projects/cdo) pour baisser la résolution spatiale à 2° et réduire la taille des fichiers :

- cdo remapbil,r180x90 era5_t_daily_2021.nc era5_t_daily_2021_2deg.nc
- ...

<div class="alert alert-warning">

<b>Transports méridiens totaux de chaleur et de quantité de mouvement zonal : </b>
$$[\overline{vT}], [\overline{uv}]$$    
    
<br>

<b>Transports méridiens de chaleur et de quantité de mouvement zonal par les perturbations transitoires : </b>
$$[\overline{u'v'}], [\overline{v'T'}]$$

<br>
    
<b>Transports méridiens de chaleur et de quantité de mouvement zonal par les circulations stationnaires non axisymétriques : </b>
$$[\overline{v^*}\overline{T^*}], [\overline{u^*}\overline{v^*}]$$

<br>

<b>Transports méridiens de chaleur et de quantité de mouvement zonal par la circulation en moyenne zonale : </b>
$$\overline{[v]} \overline{[T]}, \overline{[u]} \overline{[v]}$$
        
</div>

In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [ ]:
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_ylim(1000, 100)
    ax.set_xticks(np.arange(-90, 120, 30))
    ax.set_yticks(np.arange(1000, 0, -100))
    ax.tick_params(labelsize=8)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('Latitude')
    ax.set_ylabel('Pressure level')
    return ax

# Traitement des données

In [ ]:
year='2021'
seasons=['DJF','JJA','MAM','SON']
plev1=100
plev2=1000

In [ ]:
diri="./daily_era5/"

ft    = xr.open_dataset(diri+"era5_t_daily_2021_2deg.nc").sel(time=year).sel(level=slice(plev1,plev2))
fu    = xr.open_dataset(diri+"era5_u_daily_2021_2deg.nc").sel(time=year).sel(level=slice(plev1,plev2))
fv    = xr.open_dataset(diri+"era5_v_daily_2021_2deg.nc").sel(time=year).sel(level=slice(plev1,plev2))

lat  = fv.lat.values
lon  = fv.lon.values
lev = fv.level.values

print(fv)
print(lev)

u = fu['u']
v = fv['v']
T = ft['t']

# Fonctions pour les moyennes et les anomalies

Moyenne temporelle : $$ \overline{\alpha} = \frac{1}{T} \int_0^T \alpha dt $$

Moyenne zonale : $$ \left[ \alpha \right] = \frac{1}{2\pi} \int_0^{2\pi} \alpha d\lambda $$

Anomalie temporelle : $$ \alpha^\prime = \alpha - \overline{\alpha} $$

Anomalie zonale : $$ \alpha^* = \alpha - \left[ \alpha \right] $$

In [ ]:
def bar(data, interval=fu.time.dt.season):
    return data.groupby(interval).mean(skipna=True)

def zmean(data):
    return data.mean(dim='lon', skipna=True)

def prime(data, interval=fu.time.dt.season):
    return data.groupby(interval) - bar(data, interval=interval)

def star(data):
    return data - zmean(data)

def global_mean(data):
    return data.weighted(np.cos(np.deg2rad(data.lat))).mean(dim=('lat','lon'), skipna=True)

# Flux méridiens de chaleur et de qdm zonal

<div class="alert alert-danger">
  
<b>Calculer les flux méridiens de chaleur et de quantité de mouvement zonal : </b>
    
<br>

Transports méridiens de chaleur et de quantité de mouvement zonal total :
$$[\overline{vT}], [\overline{uv}]$$
    
<br>

Transports méridiens de chaleur et de quantité de mouvement zonal par les perturbations transitoires :
$$[\overline{u'v'}], [\overline{v'T'}]$$

<br>
    
Transports méridiens de chaleur et de quantité de mouvement zonal par les circulations stationnaires non axisymétriques :
$$[\overline{v^*}\overline{T^*}], [\overline{u^*}\overline{v^*}]$$

<br>
    
Transports méridiens de chaleur et de quantité de mouvement zonal par la circulation en moyenne zonale :
$$\overline{[v]} \overline{[T]}, \overline{[u]} \overline{[v]}$$
    
</div>

In [ ]:
vT = 
vT_transient = 
vT_stationary = 
vT_mmc = 

uv = 
uv_transient = 
uv_stationary = 
uv_mmc = 

print(vT_transient.shape)

In [ ]:
scale=np.cos(lat[np.newaxis,np.newaxis,:]*np.pi/180.)

vT=vT
vT_transient=vT_transient
vT_stationary=vT_stationary
vT_mmc=vT_mmc
uv=uv
uv_transient=uv_transient
uv_stationary=uv_stationary
uv_mmc=uv_mmc

# Flux méridiens par les transitoires

In [ ]:
levels_vT = np.arange(-15,17,2)
title = "$[\overline{v'T'}]$ : ERA5 "+year

fig = plt.figure(figsize=(15, 10))
fig.suptitle(title, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_transient.mean('season'), levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_transient.mean('season'), levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_annual_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_transient[0,:,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_transient[0,:,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_transient[1,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_transient[1,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_seasonal_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{v'T'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, vT_transient.mean('season').mean('level'), color='k', linewidth=3, label='Annual')
plt.plot(lat, vT_transient[0,:].mean('level'), label='DJF')
plt.plot(lat, vT_transient[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/eddy_heat_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
A=[]
DJF=[]
JJA=[]

for i in range(len(lat)): 
    A.append(np.trapz(vT_transient.mean('season')[:,i], x=lev)/(lev[-1]-lev[0]))
    DJF.append(np.trapz(vT_transient[0,:,i], x=lev)/(lev[-1]-lev[0]))
    JJA.append(np.trapz(vT_transient[1,:,i], x=lev)/(lev[-1]-lev[0]))

fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{v'T'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, A, color='k', linewidth=3, label='Annual')
plt.plot(lat, DJF, label='DJF')
plt.plot(lat, JJA, label='JJA')
plt.legend()

figname='./figs/eddy_heat_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
levels_uv = np.arange(-50,55,5)
title = "$[\overline{u'v'}]$ : ERA5 "+year

fig = plt.figure(figsize=(15, 10))
fig.suptitle(title, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_transient.mean('season'), levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_transient.mean('season'), levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

fig = plt.figure(figsize=(12, 12))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_transient[0,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_transient[0,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_transient[1,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_transient[1,:,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/eddy_momentum_flux_seasonal_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport ($m^2/s^2$)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{u'v'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, uv_transient.mean('season').mean('level'), color='k', linewidth=3, label='Annual')
plt.plot(lat, uv_transient[0,:].mean('level'), label='DJF')
plt.plot(lat, uv_transient[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/eddy_momentum_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
A=[]
DJF=[]
JJA=[]

for i in range(len(lat)): 
    A.append(np.trapz(uv_transient.mean('season')[:,i], x=lev)/(lev[-1]-lev[0]))
    DJF.append(np.trapz(uv_transient[0,:,i], x=lev)/(lev[-1]-lev[0]))
    JJA.append(np.trapz(uv_transient[1,:,i], x=lev)/(lev[-1]-lev[0]))

fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{u'v'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, A, color='k', linewidth=3, label='Annual')
plt.plot(lat, DJF, label='DJF')
plt.plot(lat, JJA, label='JJA')
plt.legend()

figname='./figs/eddy_momentum_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

# Flux méridiens par les circulations stationnaires non axisymétriques

In [ ]:
levels_vT = np.arange(-12,13,1)
title = "$[\overline{v^*}\overline{T^*}]$ : ERA5 "+year

fig = plt.figure(figsize=(15, 10))
fig.suptitle(title, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_stationary.mean('season'), levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_stationary.mean('season'), levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_annual_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_stationary[0,:,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_stationary[0,:,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_stationary[1,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_stationary[1,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/stat_heat_flux_seasonal_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{v^*}\overline{T^*}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, vT_stationary.mean('season').mean('level'), color='k', linewidth=3, label='Annual')
plt.plot(lat, vT_stationary[0,:].mean('level'), label='DJF')
plt.plot(lat, vT_stationary[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/stat_heat_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
A=[]
DJF=[]
JJA=[]

for i in range(len(lat)): 
    A.append(np.trapz(vT_stationary.mean('season')[:,i], x=lev)/(lev[-1]-lev[0]))
    DJF.append(np.trapz(vT_stationary[0,:,i], x=lev)/(lev[-1]-lev[0]))
    JJA.append(np.trapz(vT_stationary[1,:,i], x=lev)/(lev[-1]-lev[0]))

fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{v^*}\overline{T^*}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, A, color='k', linewidth=3, label='Annual')
plt.plot(lat, DJF, label='DJF')
plt.plot(lat, JJA, label='JJA')
plt.legend()

figname='./figs/stat_heat_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
levels_uv = np.arange(-30,35,5)
title = "$[\overline{u^*}\overline{v^*}]$ : ERA5 "+year

fig = plt.figure(figsize=(15, 10))
fig.suptitle(title, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_stationary.mean('season'), levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_stationary.mean('season'), levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/eddy_momentum_flux_annual_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_stationary[0,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_stationary[0,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_stationary[1,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_stationary[1,:,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/stat_momentum_flux_seasonal_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('$Transport (m^2/s^2)$')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{u^*}\overline{v^*}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, uv_stationary.mean('season').mean('level'), color='k', linewidth=3, label='Annual')
plt.plot(lat, uv_stationary[0,:].mean('level'), label='DJF')
plt.plot(lat, uv_stationary[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/stat_momentum_flux_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
A=[]
DJF=[]
JJA=[]

for i in range(len(lat)): 
    A.append(np.trapz(uv_stationary.mean('season')[:,i], x=lev)/(lev[-1]-lev[0]))
    DJF.append(np.trapz(uv_stationary[0,:,i], x=lev)/(lev[-1]-lev[0]))
    JJA.append(np.trapz(uv_stationary[1,:,i], x=lev)/(lev[-1]-lev[0]))

fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{u^*}\overline{v^*}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, A, color='k', linewidth=3, label='Annual')
plt.plot(lat, DJF, label='DJF')
plt.plot(lat, JJA, label='JJA')
plt.legend()

figname='./figs/stat_momentum_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

# Flux méridiens par les circulations stationnaires axisymétriques

$$\overline{[v]} \overline{[T]}, \overline{[u]} \overline{[v]}$$

In [ ]:
levels_vT = np.arange(-500,550,50)
title = "$\overline{[v]} \overline{[T]}$ : ERA5 "+year

fig = plt.figure(figsize=(15, 12))
fig.suptitle(title, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_mmc.mean('season'), levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_mmc.mean('season'), levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
 
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/zmean_heat_flux_annual_era5'
plt.savefig(figname+'.png',bbox_inches='tight')

fig = plt.figure(figsize=(12, 12))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_mmc[0,:,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_mmc[0,:,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_mmc[1,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_mmc[1,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/zmean_heat_flux_seasonal_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$\overline{[v]} \overline{[T]}$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, vT_mmc.mean('season').mean('level'), color='k', linewidth=3, label='Annual')
plt.plot(lat, vT_mmc[0,:].mean('level'), label='DJF')
plt.plot(lat, vT_mmc[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/zmean_heat_mean_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
levels_uv = np.arange(-50,55,5)
title = "$\overline{[u]} \overline{[v]}$ : ERA5 "+year

fig = plt.figure(figsize=(12, 12))
fig.suptitle(title, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_mmc.mean('season'), levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_mmc.mean('season'), levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
 
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/zmean_momentum_flux_annual_era5'
plt.savefig(figname+'.png',bbox_inches='tight')

fig = plt.figure(figsize=(12, 12))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_mmc[0,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_mmc[0,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_mmc[1,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_mmc[1,:,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/zmean_momentum_flux_seasonal_era5'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('$Transport (m^2/s^2)$')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$\overline{[u]} \overline{[v]}$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : ERA5 "+year)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, uv_mmc.mean('season').mean('level'), color='k', linewidth=3, label='Annual')
plt.plot(lat, uv_mmc[0,:].mean('level'), label='DJF')
plt.plot(lat, uv_mmc[1,:].mean('level'), label='JJA')

plt.legend()

figname='./figs/zmean_momentum_flux_era5'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()